In [59]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
data = pd.read_csv("/content/drive/MyDrive/cyber_tweets/cyberbullying_tweets.csv")

data.head(5)

data.head(5)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying


In [60]:
data['type'] = data['cyberbullying_type'].factorize()[0]
data.head(5)
from tensorflow import keras

len(data)

47692

In [77]:
import re
import string

def remove_p(text):
  translator = str.maketrans("", "", string.punctuation)
  return text.translate(translator)


import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

data['tweet_text'] = data.tweet_text.map(remove_p)

stop = set(stopwords.words('english'))

def remove_stopwords(text):
  filtered = []
  for word in text.split():
    if word.lower() not in stop:
      filtered.append(word.lower())
  return " ".join(filtered)

# Example
print(remove_stopwords(data.iloc[3]['tweet_text']))

data['tweet_text'] = data.tweet_text.map(remove_stopwords)
data.head(5)

data['type'].nunique()

from collections import Counter

def count_words(col):
  count = Counter()
  for desc in col.values:
    for word in desc.split(' '):
      count[word] += 1
  return count


count = count_words(data.tweet_text)
unique = len(count)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


jasongio meh p thanks heads concerned another angry dude twitter
/n
jasongio meh p thanks heads concerned another angry dude twitter


In [62]:
print(data['type'].nunique()
)

6


In [76]:
train_size = int(len(data) * 0.9)
train_set = data[:train_size]
test_set = data[train_size:]

train_desc = train_set.tweet_text.to_numpy()
train_labels = train_set.type.to_numpy()
test_desc = test_set.tweet_text.to_numpy()
test_labels = test_set.type.to_numpy()



In [64]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=unique)
tokenizer.fit_on_texts(train_desc)

train_sequences = tokenizer.texts_to_sequences(train_desc)
test_sequences = tokenizer.texts_to_sequences(test_desc)

from tensorflow.keras.preprocessing.sequence import pad_sequences

max_length = 30
train_padded = pad_sequences(train_sequences, maxlen = max_length, padding='post', truncating='post')
test_padded = pad_sequences(test_sequences, maxlen = max_length, padding='post', truncating='post')



In [73]:

from tensorflow.keras.layers import Dense, ReLU
from tensorflow.keras.layers import Embedding, BatchNormalization
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D, Dropout
from tensorflow.keras.models import Sequential, Model
model = Sequential()
model.add(Embedding(unique, 32, input_length=30))
model.add(Conv1D(64, 3, padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(ReLU())
model.add(Dropout(.5))
model.add(GlobalMaxPooling1D(keepdims=True))
model.add(Conv1D(64, 3, padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(ReLU())
model.add(Dropout(.5))
model.add(GlobalMaxPooling1D())
model.add(Dense(128, activation='relu'))
model.add(Dropout(.5))
model.add(Dense(6, activation='softmax'))

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["accuracy"],
)

model.fit(train_padded, train_labels, epochs=10)
model.evaluate(test_padded, test_labels)

Epoch 1/10
1342/1342 - 48s - loss: 0.9017 - accuracy: 0.6276 - 48s/epoch - 36ms/step
Epoch 2/10
1342/1342 - 49s - loss: 0.5275 - accuracy: 0.7742 - 49s/epoch - 36ms/step
Epoch 3/10
1342/1342 - 49s - loss: 0.4226 - accuracy: 0.8318 - 49s/epoch - 37ms/step
Epoch 4/10
1342/1342 - 48s - loss: 0.3494 - accuracy: 0.8715 - 48s/epoch - 36ms/step
Epoch 5/10
1342/1342 - 48s - loss: 0.2951 - accuracy: 0.8950 - 48s/epoch - 36ms/step
Epoch 6/10
1342/1342 - 47s - loss: 0.2581 - accuracy: 0.9057 - 47s/epoch - 35ms/step
Epoch 7/10
1342/1342 - 47s - loss: 0.2305 - accuracy: 0.9147 - 47s/epoch - 35ms/step
Epoch 8/10
1342/1342 - 46s - loss: 0.2086 - accuracy: 0.9218 - 46s/epoch - 35ms/step
Epoch 9/10
1342/1342 - 48s - loss: 0.1912 - accuracy: 0.9266 - 48s/epoch - 36ms/step
Epoch 10/10
1342/1342 - 47s - loss: 0.1770 - accuracy: 0.9301 - 47s/epoch - 35ms/step
150/150 - 1s - loss: 0.1801 - accuracy: 0.9595 - 524ms/epoch - 3ms/step


[0.18014217913150787, 0.9595387578010559]

In [74]:
model.evaluate(test_padded, test_labels)

150/150 [==============================] - 0s 2ms/step - loss: 0.1801 - accuracy: 0.9595


[0.18014217913150787, 0.9595387578010559]